# Assignment 1 - Die beste Gemeinde des Kantons Solothurn

In [5]:
import pandas as pd
import numpy as np
import warnings

# Suppress openpyxl warnings
warnings.simplefilter("ignore", UserWarning)

# Datenquellen:

1: Bundesamt für Statistik, Regionalporträts 2021: Kennzahlen aller Gemeinden: https://www.bfs.admin.ch/bfs/de/home/statistiken/regionalstatistik/regionale-portraets-kennzahlen/gemeinden/daten-erlaeuterungen.assetdetail.15864450.html

2: Aktueller Gemeindestand: https://www.agvchapp.bfs.admin.ch/de


In [19]:
# Load data from Gemeindestand.xlsx
file_path_gemeindestand = "data/Gemeindestand.xlsx"
df_gemeindestand = pd.read_excel(file_path_gemeindestand, sheet_name="Daten", engine="openpyxl")

# Rename columm for consistency
df_gemeindestand.rename(columns={"BFS Gde-nummer": "Gemeindecode"}, inplace=True)

#preview
df_gemeindestand.head()

,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,Gemeindecode,Gemeindename,Datum der Aufnahme
0,13256,ZH,101,Affoltern,1,Aeugst am Albis,1976-11-15
1,11742,ZH,101,Affoltern,2,Affoltern am Albis,1848-09-12
2,11801,ZH,101,Affoltern,3,Bonstetten,1848-09-12
3,11992,ZH,101,Affoltern,4,Hausen am Albis,1911-01-01
4,12249,ZH,101,Affoltern,5,Hedingen,1848-09-12


In [16]:
# Load data from Regionalportraets.xlsx
file_path_regionalportraets = "data/Regionalportraets.xlsx"

# Skip the first 8 rows and set the header to none beacuse header is spread over multiple rows
df_regionalportraets = df = pd.read_excel(
    file_path_regionalportraets,
    skiprows=5,
    header=[0, 1],
    engine="openpyxl"
)

# Drop the rows which are all NaN
df_regionalportraets.dropna(how="all", inplace=True)

# Drop the row which was not specific to a Gemeinde
df_regionalportraets = df_regionalportraets.iloc[1:].reset_index(drop=True)

# Flatten the multiindex columns
df_regionalportraets.columns = [
    col[0] if pd.isna(col[1]) else f"{col[0]}_{col[1]}"
    for col in df_regionalportraets.columns
]

#Fix the header names for Gemeindecode and Gemeindename
df_regionalportraets.rename(columns={df_regionalportraets.columns[0]: 'Gemeindecode'}, inplace=True)
df_regionalportraets.rename(columns={df_regionalportraets.columns[1]: 'Gemeindename'}, inplace=True)

# Drop the rows at the bottom that are part of the legend
df_regionalportraets = df_regionalportraets[
    pd.to_numeric(df_regionalportraets["Gemeindecode"], errors="coerce").notna()
]


#preview
df_regionalportraets.head()


,Gemeindecode,Gemeindename,Einwohner_2019,Veränderung in %_2010-2019,Bevölkerungs-dichte pro km²_2019,Ausländer in %_2019,0-19 Jahre_2019,20-64 Jahre_2019,65 Jahre und mehr_2019,Rohe Heiratssziffer_2019,...,FDP 2)_2019,CVP_2019,SP_2019,SVP_2019,EVP/CSP_2019,GLP_2019,BDP_2019,PdA/Sol._2019,GPS_2019,Kleine Rechtsparteien_2019
0,1,Aeugst am Albis,1981.0,8.607456,250.442478,14.184755,19.787986,62.190813,18.021201,7.065355,...,16.421045,2.497974,11.565102,30.380959,3.22747,14.317294,1.263724,0.16211,15.050475,3.820647
1,2,Affoltern am Albis,12303.0,10.927779,1161.756374,28.700317,20.165813,61.667886,18.166301,5.543780,...,13.385437,4.535341,14.645255,30.30692,6.618556,12.666629,2.001445,0.198718,12.590564,1.555518
2,3,Bonstetten,5572.0,8.005427,749.932705,16.564968,23.636037,59.834889,16.529074,4.316547,...,15.354984,3.867961,16.742891,26.116313,4.194018,16.237935,1.28403,0.176013,12.368531,1.385022
3,4,Hausen am Albis,3751.0,11.636905,275.808824,16.022394,22.180752,59.824047,17.995201,3.220612,...,11.135163,3.934551,13.711509,33.18478,4.059761,12.416969,1.610747,0.19312,16.744127,1.610747
4,5,Hedingen,3778.0,10.791789,578.560490,16.410799,21.625199,61.646374,16.728428,3.727370,...,13.990898,3.87366,16.738323,26.601791,3.726851,16.415344,1.881252,0.182463,12.348734,2.170676


In [23]:
 # Make sure the id is an integer
df_gemeindestand["Gemeindecode"] = df_gemeindestand["Gemeindecode"].astype(int)
df_regionalportraets["Gemeindecode"] = df_regionalportraets["Gemeindecode"].astype(int)

# Merge the two dataframes on Gemeindecode
df_merged = pd.merge(
    df_gemeindestand,
    df_regionalportraets,
    on=["Gemeindecode", "Gemeindename"],
    how="outer"
)

# Filter out all entries that are not Solothurn
df_merged = df_merged[df_merged["Kanton"] == "SO"]

df_merged.head()

,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,Gemeindecode,Gemeindename,Datum der Aufnahme,Einwohner_2019,Veränderung in %_2010-2019,Bevölkerungs-dichte pro km²_2019,...,CVP_2019,SP_2019,SVP_2019,EVP/CSP_2019,GLP_2019,BDP_2019,PdA/Sol._2019,GPS_2019,Kleine Rechtsparteien_2019,Gemeindecode_temp
792,13703.0,SO,1101.0,Gäu,2401,Egerkingen,1991-01-01,3793.0,20.950255,545.755396,...,21.639785,7.162058,28.149002,0.94086,3.955453,1.689708,*,4.781106,0.576037,2401.0
793,13704.0,SO,1101.0,Gäu,2402,Härkingen,1991-01-01,1638.0,22.972973,297.277677,...,15.564827,8.985879,39.28113,0.93068,10.077022,1.957638,*,4.525032,1.091142,2402.0
794,13705.0,SO,1101.0,Gäu,2403,Kestenholz,1991-01-01,1829.0,10.114389,212.922002,...,29.557132,11.279667,25.397602,0.78297,5.529728,1.541473,*,6.728652,0.56276,2403.0
795,13706.0,SO,1101.0,Gäu,2404,Neuendorf,1991-01-01,2246.0,18.397470,314.565826,...,28.380908,8.18905,31.586336,1.450632,5.662143,3.135236,*,4.585868,0.935891,2404.0
796,13707.0,SO,1101.0,Gäu,2405,Niederbuchsiten,1991-01-01,1246.0,29.927007,226.958106,...,30.913462,4.471154,36.923077,1.153846,4.519231,0.865385,*,3.173077,0.769231,2405.0
